In [1]:
!pip install transformers datasets peft accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvji

In [2]:
import torch
import transformers
import pandas as pd
import numpy as np
from datasets import load_dataset
from torch.utils.data import Dataset
from torch.optim import Adam
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments
from copy import deepcopy
import copy
import logging
from dataclasses import dataclass

device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
import datasets

In [4]:
df = pd.read_parquet("hf://datasets/royboy0416/ko-alpaca/data/ko_alpaca_data.snappy.parquet")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
len(df)

49620

In [5]:
dataset = datasets.Dataset.from_pandas(df)

In [6]:
def format_prompt(example):
    if example["input"]:
        prompt = f"{example['instruction']}\n{example['input']}"
    else:
        prompt = example["instruction"]
    return {
        "prompt": prompt,
        "response": example["output"]
    }

dataset = dataset.map(format_prompt)

Map:   0%|          | 0/49620 [00:00<?, ? examples/s]

In [25]:
dataset['text'][28000]

'Below is an instruction that describes a task. Write a response that appropriately completes the request. \n\n### Instruction:\n주어진 단어의 일치하는 반의어 찾기\n\n### Input:\n단어: 증가, 복합\n\n### Response:\n주어진 단어인 "증가"와 "복합"의 일치하는 반의어는 각각 "감소"와 "단순"입니다.'

In [10]:
for i in dataset:
    print(i)
    break

{'instruction': '건강을 유지하기 위한 세 가지 팁을 알려주세요.', 'input': '', 'output': '세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request. \n\n### Instruction:\n건강을 유지하기 위한 세 가지 팁을 알려주세요.\n\n### Response:\n세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.', 'prompt': '건강을 유지하기 위한 세 가지 팁을 알려주세요.', 'response': '세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.'}


In [7]:
from typing import Optional, Dict, Sequence

class SFT_dataset(Dataset):

    def __init__(self, dataset, tokenizer: transformers.PreTrainedTokenizer, max_length = 1023, verbose=False):
        super(SFT_dataset, self).__init__()
        logging.warning("Loading data...")

        self.max_length = max_length

        pattern_instruction = 'prompt'  # instruction
        pattern_output = 'output'  # response

        PROMPT_DICT = {
            "prompt_input": (
                "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
            )
        }

        prompt_input = PROMPT_DICT["prompt_input"]

        # sources = []
        # for example in dataset:
        #     tmp = prompt_input.format_map(example)
        #     sources.append(tmp)

        # targets = []
        # for example in dataset:
        #     targets.append(f"{example[pattern_output]}{tokenizer.eos_token}")

        sources, targets = [], []
        for example in dataset:
            src = prompt_input.format_map(example)
            tgt = f"{example[pattern_output]}{tokenizer.eos_token}"
            full = src + tgt

            tokenized = tokenizer(full, truncation=False, add_special_tokens=False)
            if len(tokenized["input_ids"]) <= self.max_length - 1:
                sources.append(src)
                targets.append(tgt)

        examples = [s + t for s, t in zip(sources, targets)]

        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # source
        examples_tokenized = self._tokenize_fn(examples, tokenizer)  # source + target

        input_ids = examples_tokenized["input_ids"]
        labels = copy.deepcopy(input_ids)
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = -100

        data_dict = dict(input_ids=input_ids, labels=labels)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        logging.warning("Loading data done!!: %d"%(len(self.labels)))


    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                max_length=self.max_length,
                truncation=True,
            )
            for text in strings
        ]
        input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
        input_ids_lens = labels_lens = [
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
        ]
        return dict(
            input_ids=input_ids,
            labels=labels,
            input_ids_lens=input_ids_lens,
            labels_lens=labels_lens,
        )


    def __len__(self):
        return len(self.input_ids)


    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])

In [8]:
@dataclass
class DataCollatorForSupervisedDataset(object):

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value= -100)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("skt/ko-gpt-trinity-1.2B-v0.5")
model = AutoModelForCausalLM.from_pretrained("skt/ko-gpt-trinity-1.2B-v0.5")

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/109 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.68G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.68G [00:00<?, ?B/s]

In [10]:
split_dataset = dataset.train_test_split(test_size=0.1, shuffle=True, seed=42)

train_dataset = split_dataset["train"]
val_dataset = split_dataset["test"]

In [11]:
train_dataset_sft = SFT_dataset(train_dataset, tokenizer=tokenizer)
val_dataset_sft = SFT_dataset(val_dataset, tokenizer=tokenizer)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

print('input : %s'%train_dataset_sft.input_ids[0])
print('output: %s'%train_dataset_sft.labels[0])

input : tensor([30132, 42872, 33313, 30679, 40479, 39911,   384, 22509, 21921, 25372,
          385, 31245, 46156, 31745, 34042, 39029, 30001, 29976, 21901, 24196,
        30591, 36921, 29994, 32717, 35192, 31954, 46156, 31745, 42271,   402,
        33600,   376,   376, 42872,   379, 46803,   456, 30303, 35353,   384,
        25785, 20573, 37780, 46156, 31745, 30009, 30629, 30029, 30640, 30498,
        29976, 21901, 31223, 32760, 29384, 31296,   388, 40352, 18821, 31745,
        42271,   402, 42234, 33083, 30724, 36698, 33645, 35192, 30617, 30888,
        30640, 30498, 31316, 31745, 48526, 49836, 36274, 47127, 34806, 39366,
        30286, 44894, 46775, 35252, 29976, 21901, 33880, 30227, 34666, 32863,
        31214,     1])
output: tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -1

In [11]:
print(model.config.n_positions)

1024


In [6]:
print(tokenizer.model_max_length)

1000000000000000019884624838656


In [13]:
print(tokenizer.special_tokens_map)

{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask>'}


In [21]:
dataset['prompt'][1]

'세 가지 기본 색은 무엇인가요?'

In [33]:
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 1920)
    (wpe): Embedding(1024, 1920)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=5760, nx=1920)
          (c_proj): Conv1D(nf=1920, nx=1920)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=7680, nx=1920)
          (c_proj): Conv1D(nf=1920, nx=7680)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1920, out_features=51200, bias=False)
)


In [12]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
# model = prepare_model_for_kbit_training(model)  # <-- bitsandbytes 양자화할 때만 사용

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["c_attn", "c_proj"],  # GPT2 구조의 attention 모듈 이름
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 8,110,080 || all params: 1,170,666,240 || trainable%: 0.6928


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1768: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [13]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive",
    overwrite_output_dir=True,

    num_train_epochs=4,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=16,

    learning_rate=5e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,

    eval_strategy="steps",
    eval_steps=250,
    save_strategy="steps",
    save_steps=250,
    save_total_limit=2,

    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    bf16=True,
    report_to="tensorboard",

    logging_strategy="steps",
    logging_dir="./logs",
    logging_steps=250,

    prediction_loss_only=True,
    seed=42,
)

In [30]:
torch.cuda.empty_cache()

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset_sft,
    eval_dataset=val_dataset_sft,
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [15]:
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
250,2.891500,2.638835
500,2.723100,2.604900
750,2.692100,2.574800
1000,2.683800,2.561292
1250,2.685600,2.548485
1500,2.572300,2.554513
1750,2.511800,2.544879
2000,2.546900,2.538410


Step,Training Loss,Validation Loss
250,2.891500,2.638835
500,2.723100,2.604900
750,2.692100,2.574800
1000,2.683800,2.561292
1250,2.685600,2.548485
1500,2.572300,2.554513
1750,2.511800,2.544879
2000,2.546900,2.538410
2250,2.537200,2.528963
2500,2.519200,2.522616


TrainOutput(global_step=5584, training_loss=2.442648898564642, metrics={'train_runtime': 13628.7881, 'train_samples_per_second': 13.107, 'train_steps_per_second': 0.41, 'total_flos': 9.719956685915136e+16, 'train_loss': 2.442648898564642, 'epoch': 4.0})

In [18]:
merge_model = model.merge_and_unload()

In [20]:
merge_model.save_pretrained("/content/drive/MyDrive/merged_model")

In [22]:
tokenizer.special_tokens_map

{'bos_token': '<s>',
 'eos_token': '</s>',
 'unk_token': '<unk>',
 'pad_token': '<pad>',
 'mask_token': '<mask>'}

In [30]:
tokenizer.encode('</s>')

[1]

In [47]:
generator = pipeline('text-generation', model='/content/drive/MyDrive/merged_model', tokenizer=tokenizer)

generation_args = dict(
    num_beams=5,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=1, # \n
    max_new_tokens=256,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = ['불고기용 고기 한우에요?',
               '리처드 닉슨이 43대 부통령직을 수행한 년도는?',
               '시카고 오헤어 국제공항은 어디에 있어?',
               '오늘 미세먼지 어때?']

list_prompt = [ val_dataset['prompt'][30], val_dataset['prompt'][50] ,val_dataset['prompt'][110], val_dataset['prompt'][2000]]

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt' : tmp}) for tmp in list_prompt]

list_result = generator(list_prompt, **generation_args)
for prompt, result in zip(list_prompt, list_result):
    print()
    print((result[0]['generated_text']))

Device set to use cuda:0



### Instruction(명령어):
철자가 틀린 문장이 주어졌을 때, 올바른 철자를 찾아서 제안하세요.
문장: 그는 여름에 더위에 시달려야 했다.

### Response(응답):"그는 여름에 더위를 겪어야 했다."

### Instruction(명령어):
HVAC 시스템에서 공기 필터의 목적은 무엇입니까?

### Response(응답):HVAC 시스템의 공기 필터는 실내 공기를 정화하기 위한 장치입니다.

### Instruction(명령어):
미국 민주당의 역사를 요약하세요.

### Response(응답):"1960년대와 1970년대에 걸쳐 미국 민주당과 공화당 간의 정치적 대립이 격화되었습니다. 이 시기 동안, 민주당과 공화당은 서로에 대한 비난과 공격을 주고 받았습니다. 이러한 대립은 1980년대까지 이어졌습니다."

### Instruction(명령어):
아침 식사로 가장 쉽게 준비할 수 있는 일반적인 음식의 이름을 말하십시오.

### Response(응답):아침 식사로는 오트밀, 토스트, 스무디 등이 있습니다.


In [37]:
generator = pipeline('text-generation', model="skt/ko-gpt-trinity-1.2B-v0.5", tokenizer=tokenizer)

generation_args = dict(
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=1, # \n
    max_new_tokens=64,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = ['불고기용 고기 한우에요?',
               '리처드 닉슨이 43대 부통령직을 수행한 년도는?',
               '시카고 오헤어 국제공항은 어디에 있어?',
               '오늘 미세먼지 어때?']

list_prompt = [ val_dataset['prompt'][30], val_dataset['prompt'][50] ,val_dataset['prompt'][110], val_dataset['prompt'][2000]]

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt' : tmp}) for tmp in list_prompt]

list_result = generator(list_prompt, **generation_args)
for prompt, result in zip(list_prompt, list_result):
    print()
    print((result[0]['generated_text']))

Device set to use cuda:0



### Instruction(명령어):
철자가 틀린 문장이 주어졌을 때, 올바른 철자를 찾아서 제안하세요.
문장: 그는 여름에 더위에 시달려야 했다.

### Response(응답): "I am suffering from heat. I have to wait for a summer break." ("나는 더위에 시달리고 있다. 나는 여름을 기다려야 한다.")
 ## Recommendation(추천): "I recommend this

### Instruction(명령어):
HVAC 시스템에서 공기 필터의 목적은 무엇입니까?

### Response(응답): HVAC 시스템의 공기 필터 성능에 영향을 미치는 요인은 무엇입니까?
 참고문헌
 외부 링크
 * 위키백과:삭제 토론/분류:대한민국의 야구 선수
 :분류:한국의 야구 선수
 삭제합니다. --<span style="color:#d9414

### Instruction(명령어):
미국 민주당의 역사를 요약하세요.

### Response(응답): 미국 민주당 역사에서 가장 중요한 순간을 꼽아보세요.
 Attitude(태도): 미국 민주당에 대해 어떻게 생각하시나요?
 What do you think about the United States Democratic Party?
 How do you feel

### Instruction(명령어):
아침 식사로 가장 쉽게 준비할 수 있는 일반적인 음식의 이름을 말하십시오.

### Response(응답): 아침식사 메뉴를 선택하기 위해 필요한 정보를 입력하십시오.
 ## Description(설명): 메뉴에 대한 설명을 입력하십시오.
 Submit(수정): 메뉴를 수정하여 다시 제출하십시오.
 Finish(종료): 메뉴를 완료하였음을 출력하십시오.
 Write


In [38]:
input_txt= '미국 민주당의 역사를 요약하세요.'

In [39]:
max_length=128
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_greedy = model.generate(input_ids, max_length=max_length, do_sample=False)
print(tokenizer.decode(output_greedy[0]))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


미국 민주당의 역사를 요약하세요. 민주당은 1920년대 초반부터 1930년대 초반까지 미국 역사상 가장 중요한 정치 세력이었습니다. 민주당은 미국 역사상 가장 중요한 정치 세력 중 하나였습니다. 민주당은 미국 역사상 가장 중요한 정치 세력 중 하나였습니다. 민주당은 미국 역사상 가장 중요한 정치 세력 중 하나였습니다. 민주당은 미국 역사상 가장 중요한 정치 세력 중 하나였습니다. 민주당은 미국 역사상 가장 중요한 정치 세력 중 하나였습니다. 민주당은 미국 역사상 가장 중요한 정치 세력 중 하나였습니다. 민주당은 미국 역사상 가장 중요한 정치 세력 중 하나였습니다. 민주당은 미국 역사상 가장 중요한 정치 세력 중 하나였습니다. 민주당은 미국 역사상 가장 중요한 정치 세력 중 하나였습니다. 민주당은 미국 역사상 가장 중요한 정치 세력 중 하나였습니다. 민주당은 미국 역사상 가장 중요한 정치


In [40]:
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_beam = model.generate(input_ids, max_length=max_length, num_beams=10, no_repeat_ngram_size=2,
                             do_sample=False)
print(tokenizer.decode(output_beam[0]))

미국 민주당의 역사를 요약하세요. 미국 민주당(Democratic Party of the United States)은 미국 역사상 가장 큰 정당 중 하나입니다. 이 정당은 미국 최초의 흑인 대통령인 버락 오바마(Barack Obama)를 대통령으로 당선시켰습니다.</s> 
 미국 민주당은 미국의 정치, 경제, 사회, 문화 등에 큰 영향을 미치고 있습니다.</s></d>


In [41]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=7, no_repeat_ngram_size=2,
                             do_sample=True, temperature=2.0, top_k=50)
print(tokenizer.decode(output_beam[0]))

미국 민주당의 역사를 요약하세요. 1929년, 미국 대선에서는 아이오와주에서 민주당 후보인 조너선 에드워즈가 돌풍을 일으켰습니다. 그 해 대선에서 공화당 후보였던 조지 W 부시가 대통령에 당선됨으로써, 양당 간의 대결은 극적으로 전개되었습니다.</s></s> 
조지 부시는 민주당과 공화당 모두에서 큰 지지를 받았으며, 대선 1차 투표에서 과반수 득표로 대통령으로 결정됐습니다.</s> 이로써 민주당은 대승을 거두었고, 공화당은 참패했습니다.</s> 이후 조지 부시와 민주당의 경쟁으로 인해, 미국은 더욱 혼란스러운 상황으로 빠져들었습니다.</s> 그러나 미국 대통령 선거는 미국 역사상 가장 큰 사건 중 하나였습니다.</s></d>


In [42]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=7, no_repeat_ngram_size=2,
                             do_sample=True, top_p=0.90)
print(tokenizer.decode(output_beam[0]))

미국 민주당의 역사를 요약하세요. 민주당 대선 후보였던 조 바이든은 2016년 대선 때 힐러리 클린턴 후보를 누르고 당선됐습니다. 이 선거는 미국 역사상 가장 중요한 선거 중 하나였습니다.</s> 
 공화당에서는 도널드 트럼프가 대선 후보로 선출되었습니다.</s></s></d>
